In [4]:
#import required libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, models

In [5]:
data = pd.read_csv('creditcard.csv')

# Split the data into features and labels
X = data.drop(['Class'], axis=1)
y = data['Class']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
input_dim = X_train.shape[1]
encoding_dim = 14
hidden_dim1=int(encoding_dim/2)
hidden_dim2=4

input_layer = tf.keras.Input(shape=(input_dim, ))
encoder = tf.keras.layers.Dense(encoding_dim, activation="tanh")(input_layer)
encoder = tf.keras.layers.Dropout(0.2)(encoder)
encoder = tf.keras.layers.Dense(hidden_dim1, activation='relu')(encoder)
encoder = tf.keras.layers.Dense(hidden_dim2, activation=tf.nn.leaky_relu)(encoder)

decoder = tf.keras.layers.Dense(hidden_dim1, activation='relu')(encoder)
decoder = tf.keras.layers.Dropout(0.2)(decoder)
decoder = tf.keras.layers.Dense(encoding_dim, activation='relu')(decoder)
decoder = tf.keras.layers.Dense(input_dim, activation='tanh')(decoder)

autoencoder = tf.keras.Model(inputs=input_layer, outputs=decoder)
autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 30)]              0         
                                                                 
 dense_1 (Dense)             (None, 14)                434       
                                                                 
 dropout_1 (Dropout)         (None, 14)                0         
                                                                 
 dense_2 (Dense)             (None, 7)                 105       
                                                                 
 dense_3 (Dense)             (None, 4)                 32        
                                                                 
 dense_4 (Dense)             (None, 7)                 35        
                                                                 
 dropout_2 (Dropout)         (None, 7)                 0     

In [8]:
# 5. Compile the models with Optimizer, Loss, and Evaluation Metrics
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train the autoencoder
autoencoder.fit(X_train, X_train, epochs=10, batch_size=32, shuffle=True, validation_data=(X_test, X_test))

# Predict using the autoencoder
reconstructed = autoencoder.predict(X_test)

# Calculate the Mean Squared Error (MSE) between original and reconstructed data
mse = np.mean(np.power(X_test - reconstructed, 2), axis=1)

# Define a threshold for anomaly detection
threshold = np.mean(mse) + 2 * np.std(mse)

# Classify anomalies based on the threshold
y_pred = (mse > threshold).astype(int)

# Evaluate the model
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print("Accuracy: {:.2f}%".format(accuracy_score(y_test, y_pred) * 100))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Epoch 1/10
7121/7121 [==============================] - 6s 846us/step - loss: 0.8842 - val_loss: 0.8145
Epoch 2/10
7121/7121 [==============================] - 6s 832us/step - loss: 0.8428 - val_loss: 0.7973
Epoch 3/10
7121/7121 [==============================] - 6s 832us/step - loss: 0.8326 - val_loss: 0.7909
Epoch 4/10
7121/7121 [==============================] - 6s 831us/step - loss: 0.8274 - val_loss: 0.7864
Epoch 5/10
7121/7121 [==============================] - 6s 831us/step - loss: 0.8250 - val_loss: 0.7848
Epoch 6/10
7121/7121 [==============================] - 6s 830us/step - loss: 0.8237 - val_loss: 0.7833
Epoch 7/10
7121/7121 [==============================] - 6s 831us/step - loss: 0.8228 - val_loss: 0.7835
Epoch 8/10
7121/7121 [==============================] - 6s 831us/step - loss: 0.8218 - val_loss: 0.7821
Epoch 9/10
7121/7121 [==============================] - 6s 833us/step - loss: 0.8210 - val_loss: 0.7829
Epoch 10/10
1781/1781 [==============================] - 1s 428u